<h1 style="text-align: center;">
    Assignment 3: Object Detection
</h1>
<div style="text-align: right;"> <b>
    Name: Pronoy Patra <br>
    Roll No.: 2021112019
</div>


# Q1: Face detection and association-based tracking

### 1. Data preparation. 

In [2]:
!pip install decord

     ---------------------------------------- 0.0/24.7 MB ? eta -:--:--
     ---------------------------------------- 0.1/24.7 MB 1.1 MB/s eta 0:00:23
     ---------------------------------------- 0.1/24.7 MB 1.2 MB/s eta 0:00:21
     ---------------------------------------- 0.2/24.7 MB 1.5 MB/s eta 0:00:17
      --------------------------------------- 0.3/24.7 MB 1.8 MB/s eta 0:00:14
      --------------------------------------- 0.4/24.7 MB 1.6 MB/s eta 0:00:16
      --------------------------------------- 0.5/24.7 MB 1.8 MB/s eta 0:00:14
     - -------------------------------------- 0.7/24.7 MB 2.0 MB/s eta 0:00:13
     - -------------------------------------- 0.8/24.7 MB 2.2 MB/s eta 0:00:11
     - -------------------------------------- 1.0/24.7 MB 2.3 MB/s eta 0:00:11
     - -------------------------------------- 1.1/24.7 MB 2.4 MB/s eta 0:00:10
     - -------------------------------------- 1.2/24.7 MB 2.3 MB/s eta 0:00:11
     -- ------------------------------------- 1.2/24.7 MB 2


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import decord
import cv2

def extract_frames(video_path, output_folder, duration=30):
    video = decord.VideoReader(video_path)
    
    num_frames = int(video.get_avg_fps() * duration)
    print("Expected number of frames is: ", num_frames)
    extracted_frames = video.get_batch(range(num_frames)).asnumpy()
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for i, frame in enumerate(extracted_frames):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imwrite(os.path.join(output_folder, f"frame_{i+1:04d}.jpg"), frame_rgb)

video_path = "q1.mp4"
output_folder = "frames_output"
extract_frames(video_path, output_folder)


Expected number of frames is:  719


We can use decord. We first find the average frames pre second, and then use it get the estimated number of frames in the initial 30 seconds. These can then be extracted and saved in a folder. 

### 2. Face detection.

In [7]:
import os
import cv2
import time

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # print("hey")
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4) # 5, minSize=(30, 30)
    # print("there")
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    return frame

def process_frames(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    files = os.listdir(input_folder)
    
    for filename in files:
        if filename.endswith(".jpg"):
            frame = cv2.imread(os.path.join(input_folder, filename))
            start_time = time.time()
            frame_with_faces = detect_faces(frame)
            
            cv2.imwrite(os.path.join(output_folder, filename), frame_with_faces)
            
            elapsed_time = time.time() - start_time
            print(f"Processed {filename} in {elapsed_time:.4f} seconds")

input_folder = "frames_output"
output_folder = "processed_frames_output"

process_frames(input_folder, output_folder)


Processed frame_0001.jpg in 0.0735 seconds
Processed frame_0002.jpg in 0.0795 seconds
Processed frame_0003.jpg in 0.0771 seconds
Processed frame_0004.jpg in 0.0774 seconds
Processed frame_0005.jpg in 0.0681 seconds
Processed frame_0006.jpg in 0.0683 seconds
Processed frame_0007.jpg in 0.0731 seconds
Processed frame_0008.jpg in 0.0744 seconds
Processed frame_0009.jpg in 0.0706 seconds
Processed frame_0010.jpg in 0.0722 seconds
Processed frame_0011.jpg in 0.0615 seconds
Processed frame_0012.jpg in 0.0726 seconds
Processed frame_0013.jpg in 0.0811 seconds
Processed frame_0014.jpg in 0.0997 seconds
Processed frame_0015.jpg in 0.1040 seconds
Processed frame_0016.jpg in 0.0962 seconds
Processed frame_0017.jpg in 0.0734 seconds
Processed frame_0018.jpg in 0.0717 seconds
Processed frame_0019.jpg in 0.0756 seconds
Processed frame_0020.jpg in 0.0901 seconds
Processed frame_0021.jpg in 0.0750 seconds
Processed frame_0022.jpg in 0.0796 seconds
Processed frame_0023.jpg in 0.0779 seconds
Processed f

Viola-Jones Haar cascades based face detector from OpenCV to detect faces in each frame. On average around 0.06 to 0.09 seconds are taken to process each frame. 

The processing time for each frame when using the Viola-Jones Haar cascades based face detector from OpenCV can vary depending on several factors:

<b>Resolution of the Frame:</b> Higher resolution frames will take longer to process since there are more pixels to analyze. Reducing the resolution can significantly decrease processing time.

<b>Number of Faces in the Frame:</b> The more faces there are in a frame, the longer it will take to detect them all. This is because the algorithm needs to search for potential face regions across the entire frame.

<b>Optimization Techniques:</b> OpenCV may employ various optimization techniques to speed up face detection, such as parallel processing, SIMD instructions, and algorithmic optimizations. The effectiveness of these techniques can influence processing time.

<b>Region of Interest (ROI) Selection:</b> Limiting the search for faces to specific regions of interest within the frame can reduce processing time, especially if the application has prior knowledge of where faces are likely to appear.y to appear.

### 3. Face detection visualization.

In [9]:
import cv2
import os

def images_to_video(image_folder, output_video, frame_rate=30):
    images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
    images.sort()

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (width, height))

    for image in images:
        img_path = os.path.join(image_folder, image)
        frame = cv2.imread(img_path)
        video.write(frame)

    cv2.destroyAllWindows()
    video.release()
    print("Video created")

image_folder = "./processed_frames_output"
output_video = "new_q1.mp4"
frame_rate = 24
images_to_video(image_folder, output_video, frame_rate)


Video created


<b>Link to the video:</b> https://drive.google.com/file/d/1jQa5xgbuVcgf9c1Sim6yHkZwl-lZ6Q9K/view?usp=sharing

Some possible conclusions could be:

<b>Successful Detection:</b> The face detector correctly identifies and draws bounding boxes around faces when they are well-lit, facing the camera directly, and have sufficient contrast with the background.

<b>Failure in Low Light Conditions:</b> The face detector might fail to detect faces in low light conditions where there isn't enough contrast between the face and the background, resulting in missed detections or false positives.

<b>Challenges with Pose and Occlusion:</b> The face detector may struggle with faces that are partially occluded or in non-frontal poses. It might miss or incorrectly detect faces in these situations.

This is the case because there can be missed detections causing the missing cases of tracking where actually it is continuous. 

### 4. Association-based tracking.

In [31]:
import cv2
import os

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    intersect_x1 = max(x1, x2)
    intersect_y1 = max(y1, y2)
    intersect_x2 = min(x1 + w1, x2 + w2)
    intersect_y2 = min(y1 + h1, y2 + h2)

    intersect_area = max(0, intersect_x2 - intersect_x1 + 1) * max(0, intersect_y2 - intersect_y1 + 1)

    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate IoU
    iou = intersect_area / float(box1_area + box2_area - intersect_area)

    return iou

# Function to track faces across frames
def track_faces(video_path, output_folder, duration=30):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    num_frames = int(fps * duration)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    track_id = 0
    face_tracks = {}

    for i in range(num_frames):
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for track_id, bbox in face_tracks.items():
            x, y, w, h = bbox
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, str(track_id), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # New face tracks or update existing ones based on IoU
        for (x, y, w, h) in faces:
            matched = False
            for track_id, bbox in face_tracks.items():
                iou = calculate_iou((x, y, w, h), bbox)
                if iou > 0.5:
                    face_tracks[track_id] = (x, y, w, h)
                    matched = True
                    break
            if not matched:
                face_tracks[track_id + 1] = (x, y, w, h)
                track_id += 1

        cv2.imwrite(os.path.join(output_folder, f"frame_{i:04d}.jpg"), frame)

    cap.release()
    cv2.destroyAllWindows()

    num_tracks = len(face_tracks)
    print("Number of unique tracks:", num_tracks)

video_path = 'q1.mp4'
output_folder = 'annotated_frames'
track_faces(video_path, output_folder, duration=30)


Number of unique tracks: 60


In [33]:
import cv2
import os

def images_to_video(image_folder, output_video, frame_rate=30):
    images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
    images.sort()

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (width, height))

    for image in images:
        img_path = os.path.join(image_folder, image)
        frame = cv2.imread(img_path)
        video.write(frame)

    cv2.destroyAllWindows()
    video.release()
    print("Video created")

image_folder = "./annotated_frames"
output_video = "new_q1_annotate.mp4"
frame_rate = 24
images_to_video(image_folder, output_video, frame_rate)


Video created


<b>Link to the video: </b> https://drive.google.com/file/d/1VBtTK3O3zc8SWY6CwVe0xrdOpdZzEOrj/view?usp=sharing

There are multiple failure cases that can be observed during tracking using this method. 

Incorrect associations between faces or the inability to track a face across multiple frames. 

Missed detections result in creation of multiple other tracks which are not actually true. 

# Q2: YOLO Object Detection

### 1. Data preparation. 

Downloaded the data consisting of images and labels folder. 
    
Each consisted of two folder train and val. 
    
The names of the images and their labels are same. They just have different extensions. 

### 2. Understanding YOLO object detector.

Difference between Single-shot Detectors (e.g., YOLO) and Region-based Detectors (e.g., R-CNN series):

Model Architecture:

Single-shot detectors (e.g., YOLO): YOLO directly predicts bounding boxes and class probabilities for multiple objects within a single network evaluation. It divides the input image into a grid and predicts bounding boxes and class probabilities directly from each grid cell. This enables YOLO to be faster since it only requires a single forward pass through the network.
Region-based detectors (e.g., R-CNN series): Region-based detectors first propose a set of candidate object regions (region proposals) using a separate algorithm or module. These proposals are then passed through a CNN to extract features, followed by a classification step to determine the object class and a refinement step to adjust the bounding box coordinates. This multi-step process makes region-based detectors slower compared to single-shot detec
tors.
Speed and Efficiency:

Single-shot detectors (e.g., YOLO): YOLO is generally faster than region-based detectors because it processes the entire image in a single pass through the network. This makes it more suitable for real-time applications or scenarios where speed is critical.
Region-based detectors (e.g., R-CNN series): Region-based detectors are slower than single-shot detectors because they involve multiple stages, including region proposal generation, feature extraction, classification, and bounding box refinement. This makes them less efficient for real-time applications but often results in higher accuracy.
Localization Accuracy:

Single-shot detectors (e.g., YOLO): YOLO may suffer from lower localization accuracy compared to region-based detectors, especially for small objects or objects with intricate shapes. Since YOLO predicts bounding boxes directly from grid cells, it may struggle to precisely localize objects with fine details.
Region-based detectors (e.g., R-CNN series): Region-based detectors tend to achieve higher localization accuracy because they use region proposals to localize objects more precisely. By refining the bounding box coordinates based on the extracted features, region-based detectors can better capture the spatial detail

Differences between Three Versions of YOLO Series:

YOLOv3:

YOLOv3 introduced multiple improvements over its predecessors, including a three-stage detection process, feature pyramid network (FPN) integration, and use of anchor boxes for bounding box prediction. These enhancements helped improve both accuracy and speed compared to previous versions.
It divides the input image into a grid and predicts bounding boxes and class probabilities using a combination of convolutional layers at three different scales, enabling it to detect objects of varying sizes.
YOLOv3 also introduced the concept of Darknet-53, a deeper CNN architecture, for feature extraction.
YOLOv4:

YOLOv4 further improved upon the speed and accuracy of YOLOv3 by introducing a series of architectural enhancements and optimization techniques. These include the integration of CSPDarknet53 for feature extraction, improved training strategies like mosaic data augmentation and self-adversarial training, and utilization of various network components like SPP (Spatial Pyramid Pooling) and PAN (Path Aggregation Network) modules.
YOLOv4 also introduced the use of weighted residual connections and cross-stage partial connections (CSP) to improve gradient flow and information propagation through the network.
YOLOv5:

YOLOv5, developed by Ultralytics, introduced a streamlined and simplified architecture compared to previous versions. It utilizes a CSPNet backbone combined with SPP, PAN, and C3 modules for feature extraction and feature fusion.
YOLOv5 also introduced a novel training pipeline that includes automatic hyperparameter tuning and advanced data augmentation techniques like mosaic data augmentation, mixup, and cutmix. These enhancements contribute to improved performance and faster convergence during training.
YOLOv5 is available in various sizes (e.g., YOLOv5s, YOLOv5m, YOLOv5l, YOLOv5x), allowing users to choose the model size based on their specific requirements for speed and accuracy.s of objects.

### 3. Hands on with ultralytics. 

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import torch

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/78.2 GB disk)


In [4]:
model_n = YOLO('yolov8n.yaml')
params = sum(p.numel() for p in model_n.parameters())
print("Parameters in YOLOv8n Nano model:", params)
conv_layers = sum(1 for m in model_n.modules() if isinstance(m, torch.nn.Conv2d))
print("Convolutional layers in YOLOv8n Nano model:", conv_layers)

model_m = YOLO('yolov8m.yaml')
params = sum(p.numel() for p in model_m.parameters())
print("Parameters in YOLOv8m Nano model:", params)
conv_layers = sum(1 for m in model_m.modules() if isinstance(m, torch.nn.Conv2d))
print("Convolutional layers in YOLOv8m Nano model:", conv_layers)


Parameters in YOLOv8n Nano model: 3157200
Convolutional layers in YOLOv8n Nano model: 64
Parameters in YOLOv8m Nano model: 25902640
Convolutional layers in YOLOv8m Nano model: 84


As we observe that YOLOv8m Nano model has greater number of parameters and convolutional layers than YOLOv8n Nano model.

### 4. Training YOLO variants.

In [5]:
import os
import random
import shutil

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/Colab Notebooks'

train1_images_dir = os.path.join(root_dir, 'images', 'train1')
os.makedirs(train1_images_dir, exist_ok=True)

train1_labels_dir = os.path.join(root_dir, 'labels', 'train1')
os.makedirs(train1_labels_dir, exist_ok=True)

image_files = os.listdir(os.path.join(root_dir, 'images', 'train'))

selected_images = random.sample(image_files, 100)

for image in selected_images:
    src_image = os.path.join(root_dir, 'images', 'train', image)
    dst_image = os.path.join(train1_images_dir, image)
    shutil.copy(src_image, dst_image)

    label_file = image.replace('.png', '.txt') 
    src_label = os.path.join(root_dir, 'labels', 'train', label_file)
    dst_label = os.path.join(train1_labels_dir, label_file)
    shutil.copy(src_label, dst_label)

print("100 random images and their corresponding labels have been copied to train1 folder.")


In [6]:
print("For yolov8n initialized from scratch and train1 (100 images)")
model = YOLO('yolov8n.yaml')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_100.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


For yolov8n initialized from scratch and train1 (100 images)
Mounted at /content/drive
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/Colab Notebooks/config_100.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sho

100%|██████████| 755k/755k [00:00<00:00, 110MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 144MB/s]


AMP: checks passed ✅


...


20 epochs completed in 0.035 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.94it/s]


                   all         50         52    0.00232      0.635     0.0439     0.0137
Speed: 0.3ms preprocess, 3.3ms inference, 0.0ms loss, 5.4ms postprocess per image
Results saved to runs/detect/train
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]


                   all         50         52    0.00245      0.635     0.0509     0.0152
Speed: 0.3ms preprocess, 33.4ms inference, 0.0ms loss, 4.4ms postprocess per image
Results saved to runs/detect/train2
AP50 Train:  0.04391571976833551
AP50 Validation:  0.05087864089044355


In [7]:
print("For yolov8n from pretrained weights and train1 (100 images)")
model = YOLO('yolov8n.pt')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_100.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


For yolov8n from pretrained weights and train1 (100 images)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/config_100.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, 

train: Scanning /content/drive/MyDrive/Colab Notebooks/labels/train1.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.025 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.86it/s]


                   all         50         52      0.712      0.558      0.627      0.446
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/train3
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.09s/it]


                   all         50         52      0.702      0.538      0.624      0.444
Speed: 0.7ms preprocess, 16.1ms inference, 0.0ms loss, 7.5ms postprocess per image
Results saved to runs/detect/train32
AP50 Train:  0.6266877296584789
AP50 Validation:  0.6239444403026005


In [8]:
print("For yolov8m from pretrained weights and train1 (100 images)")
model = YOLO('yolov8m.pt')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_100.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


For yolov8m from pretrained weights and train1 (100 images)


100%|██████████| 49.7M/49.7M [00:00<00:00, 280MB/s]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Colab Notebooks/config_100.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=

train: Scanning /content/drive/MyDrive/Colab Notebooks/labels/train1.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...


20 epochs completed in 0.043 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 52.0MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.51it/s]


                   all         50         52      0.759      0.423      0.599      0.514
Speed: 0.2ms preprocess, 9.5ms inference, 0.0ms loss, 5.4ms postprocess per image
Results saved to runs/detect/train4
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.30s/it]


                   all         50         52      0.426      0.712      0.614      0.509
Speed: 0.2ms preprocess, 37.5ms inference, 0.0ms loss, 7.8ms postprocess per image
Results saved to runs/detect/train42
AP50 Train:  0.5989925125850402
AP50 Validation:  0.6141739967947338


In [9]:
print("For yolov8n initialized from scratch and train (400 images)")
model = YOLO('yolov8n.yaml')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_400.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


For yolov8n initialized from scratch and train (400 images)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/Colab Notebooks/config_400.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False

train: Scanning /content/drive/MyDrive/Colab Notebooks/labels/train.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train5
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.098 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.15it/s]


                   all         50         52      0.312      0.288      0.263      0.103
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train5
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.01s/it]


                   all         50         52      0.313      0.288      0.263      0.103
Speed: 0.4ms preprocess, 9.4ms inference, 0.0ms loss, 6.8ms postprocess per image
Results saved to runs/detect/train52
AP50 Train:  0.26290280944684785
AP50 Validation:  0.26318595880448464


In [10]:
print("For yolov8n from pretrained weights and train (400 images)")
model = YOLO('yolov8n.pt')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_400.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


For yolov8n from pretrained weights and train (400 images)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/config_400.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, v

train: Scanning /content/drive/MyDrive/Colab Notebooks/labels/train.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train6
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.102 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.2MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.69it/s]


                   all         50         52       0.68      0.694       0.69      0.515
Speed: 0.2ms preprocess, 3.9ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/train6
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


                   all         50         52      0.663      0.682      0.691      0.526
Speed: 0.7ms preprocess, 11.7ms inference, 0.0ms loss, 4.5ms postprocess per image
Results saved to runs/detect/train62
AP50 Train:  0.6895872655088289
AP50 Validation:  0.6907874068612142


In [11]:
print("For yolov8m from pretrained weights and train (400 images)")
model = YOLO('yolov8m.pt')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_400.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


For yolov8m from pretrained weights and train (400 images)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
New https://pypi.org/project/ultralytics/8.1.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Colab Notebooks/config_400.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0

train: Scanning /content/drive/MyDrive/Colab Notebooks/labels/train.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train7
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.145 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 52.0MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.53it/s]


                   all         50         52      0.612      0.712      0.645      0.509
Speed: 0.2ms preprocess, 10.7ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train7
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.13it/s]


                   all         50         52       0.62      0.673      0.643      0.504
Speed: 1.0ms preprocess, 28.3ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/train72
AP50 Train:  0.6452569186130865
AP50 Validation:  0.6434129485741187


In [38]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from google.colab import drive
drive.mount('/content/drive')

# Path to the root directory
root_dir = '/content/drive//MyDrive/Colab Notebooks'

# Path to the validation images directory
val_images_dir = os.path.join(root_dir, 'images', 'val')

# List all image files in the validation images directory
image_files = os.listdir(val_images_dir)

# Select four random image files
selected_images = random.sample(image_files, 4)

images = [plt.imread(os.path.join(val_images_dir, image)) for image in selected_images]

results = model([os.path.join(val_images_dir, image) for image in selected_images])

# Process results list
for i, result in enumerate(results):
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.show()  # display to screen
    result.save(filename=f'result{i}.jpg')  # save to disk

    print(boxes.xyxy)
    # print(boxes.xywh, boxes.xywhn, boxes.xyxy, boxes.xyxyn)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

0: 640x640 1 duck, 30.7ms
1: 640x640 2 ducks, 30.7ms
2: 640x640 1 duck, 30.7ms
3: 640x640 1 duck, 30.7ms
Speed: 3.6ms preprocess, 30.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
tensor([[200.3196, 205.1212, 776.1187, 427.6102]], device='cuda:0')
tensor([[ 468.8987,  237.8738, 1010.2822,  604.7591],
        [ 170.7801,  227.1279, 1008.9236,  617.0688]], device='cuda:0')
tensor([[196.3700, 354.3766, 489.9339, 568.0363]], device='cuda:0')
tensor([[430.1649, 236.6491, 865.5407, 613.9747]], device='cuda:0')


### 5. Impact of augmentations.

In [51]:
print("For yolov8n from pretrained weights and train1 (100 images)")
model = YOLO('yolov8n.pt')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_100.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


For yolov8n from pretrained weights and train1 (100 images)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
New https://pypi.org/project/ultralytics/8.1.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/config_100.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=

train: Scanning /content/drive/MyDrive/Colab Notebooks/labels/train1.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train8/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train8
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.026 hours.
Optimizer stripped from runs/detect/train8/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train8/weights/best.pt, 6.2MB

Validating runs/detect/train8/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.77it/s]


                   all         50         52      0.712      0.558      0.627      0.446
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/detect/train8
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]


                   all         50         52      0.702      0.538      0.624      0.444
Speed: 0.5ms preprocess, 9.0ms inference, 0.0ms loss, 9.4ms postprocess per image
Results saved to runs/detect/train82
AP50 Train:  0.6266877296584789
AP50 Validation:  0.6239444403026005


In [4]:
!pip3 uninstall -y albumentations

Found existing installation: albumentations 1.3.1
Uninstalling albumentations-1.3.1:
  Successfully uninstalled albumentations-1.3.1


In [6]:
print("For yolov8n initialized from scratch and train1 (100 images)")
model = YOLO('yolov8n.yaml')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_100.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


For yolov8n initialized from scratch and train1 (100 images)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/Colab Notebooks/config_100.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fals

train: Scanning /content/drive/MyDrive/Colab Notebooks/labels/train1.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      2.31G      3.398      4.238       4.31         11        640: 100%|██████████| 7/7 [00:03<00:00,  2.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.74it/s]

                   all         50         52    0.00218      0.615    0.00533    0.00185


...


20 epochs completed in 0.022 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.03it/s]


                   all         50         52    0.00305      0.538     0.0241    0.00563
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/train3
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.41it/s]


                   all         50         52    0.00313      0.538     0.0268    0.00453
Speed: 0.3ms preprocess, 10.2ms inference, 0.0ms loss, 6.8ms postprocess per image
Results saved to runs/detect/train32
AP50 Train:  0.024139907828765662
AP50 Validation:  0.026810685992029903


In [5]:
print("For yolov8n from pretrained weights and train1 (100 images)")
model = YOLO('yolov8n.pt')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_100.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


For yolov8n from pretrained weights and train1 (100 images)
Mounted at /content/drive
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/config_100.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=F

100%|██████████| 755k/755k [00:00<00:00, 25.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/Colab Notebooks/labels/train1.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.032 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.80it/s]


                   all         50         52      0.804      0.654      0.679      0.465
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/train
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.03it/s]


                   all         50         52      0.813      0.635      0.679      0.465
Speed: 0.3ms preprocess, 42.5ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs/detect/train2
AP50 Train:  0.6792503872690573
AP50 Validation:  0.6790155151796338


Yes, the default run of YOLOv5 by Ultralytics uses augmentations during training. Augmentations are transformations applied to the training images to artificially increase the diversity of the dataset and improve the model's generalization capability. The specific augmentations used may vary depending on the configuration and version of YOLOv5, but some common augmentations include:

Random Horizontal Flip: Flips the image horizontally with a certain probability. This helps the model become invariant to left-right orientation.

Random Vertical Flip: Similar to horizontal flip but flips the image vertically.

Random Rotation: Rotates the image by a random angle within a specified range. This helps in making the model invariant to rotation.

Random Resize and Crop: Randomly resizes and crops the image to a specified size. This helps in handling variations in object sizes and positions.

Random Brightness and Contrast Adjustment: Adjusts the brightness and contrast of the image randomly. This helps in making the model robust to different lighting conditions.

Random Saturation and Hue Adjustment: Adjusts the saturation and hue of the image randomly. This helps in making the model invariant to color variations.

Random Gaussian Blur: Applies Gaussian blur to the image with a random kernel size. This helps in simulating blur in real-world images.

Random Noise Injection: Adds random noise to the image. This helps in making the model robust 

Random Horizontal Flip, Random Resize and Crop, Random Rotation, Random Brightness and Contrast Adjustment augmentations are often considered important because they address common challenges in object detection tasks, such as scale variation, orientation variation, lighting variation, and color variation. By incorporating these augmentations into the training process, the model becomes more robust and capable of generalizing well to unseen data.to noisy images.

In [ ]:
print("For yolov8m from pretrained weights and train1 (100 images)")
model = YOLO('yolov8m.pt')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_100.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


In [ ]:
print("For yolov8n initialized from scratch and train (400 images)")
model = YOLO('yolov8n.yaml')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_400.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


In [ ]:
print("For yolov8n from pretrained weights and train (400 images)")
model = YOLO('yolov8n.pt')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_400.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)


In [ ]:
print("For yolov8m from pretrained weights and train (400 images)")
model = YOLO('yolov8m.pt')

from google.colab import drive
drive.mount('/content/drive')

results= model.train(data='/content/drive/MyDrive/Colab Notebooks/config_400.yaml', epochs=20)
metrics = model.val()

print("AP50 Train: ", results.results_dict['metrics/mAP50(B)'])
print("AP50 Validation: ", metrics.box.map50)
